In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import datetime,gc,math
import random
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.metrics import recall_score, auc, accuracy_score, f1_score, precision_score, classification_report, roc_auc_score,mean_squared_error
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
%matplotlib inline

In [2]:
def active_time_transform(df):
    df['role_created_active_time'].fillna('"0-8,0","8-12,0","12-14,0","14-18,0","18-24,0"', inplace=True)
    df['role_created_active_time'] = df['role_created_active_time'].apply(lambda x:str(x)[1:-1])
    temp = df['role_created_active_time'].str.split(',',expand=True).iloc[:,[1,3,5,7,9]].rename(columns={1:'active_0-8',
                                        3:'active_8-12',5:'active_12-14',7:'active_14-18',9:'active_18-24'})
    for i in range(len(temp.columns)):
        temp.iloc[:,i] = temp.iloc[:,i].apply(lambda x:str(x)[:-1])
    df = df.join(temp).drop(['role_created_active_time'], axis=1)
    return df

def pay_grade_transform(df):
    df['pay_grade'].fillna('[0,0,0,0,0,0,0]',inplace=True)
    df['pay_grade'] = df['pay_grade'].apply(lambda x:str(x)[1:-1])
    temp = df['pay_grade'].str.split(',',expand=True).rename(columns={0:'pay_grade_1',
                1:'pay_grade_2',2:'pay_grade_3',3:'pay_grade_4',4:'pay_grade_5',5:'pay_grade_6',6:'pay_grade_7'})
    df = df.join(temp).drop(['pay_grade'], axis=1)
    return df

In [3]:
%%time
role_info = pd.read_csv('./data/mr_role_3d.csv')
role_info.drop_duplicates(subset=['user_id','cp_server_no','cp_role_id'],inplace=True)
role_info = active_time_transform(role_info)
role_info = pay_grade_transform(role_info)
role_info.to_pickle('./role_info_3d.pickle')

Wall time: 1min 55s


In [4]:
# role_info = pd.read_pickle('./role_info_3d.pickle')
# 删除2020-7-19日前30天的数据
role_info['create_role_time'] = pd.to_datetime(role_info['create_role_time'], format='%Y-%m-%d %H:%M:%S')
role_info.drop(role_info[role_info['create_role_time'] > (role_info['create_role_time'].max() + 
                                                          datetime.timedelta(-30))].index,inplace=True)

In [5]:
pd.set_option('display.max_columns',None)

In [6]:
role_info.head()

,user_id,mgame_id,cp_server_no,cp_role_id,create_role_time,role_name,role_id,p_model,model_money,sys_ver,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,device_num,city_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_grade_7
0,96859161,1056,13313637,13313637,2020-02-16 15:48:52,13313637,11436054,iPhone 6s Plus,0.00,12.4.1,1.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0
1,20332688,1056,13542741,13542741,2020-02-12 20:10:30,無極,11430635,iPhone XR,3828.66,12.2,19.0,2.0,3008.0,1.0,2.0,1.0,1.0,NaN,NaN,0,0,0,15,14,0,0,0,0,0,0,0
5,99489283,1056,13697851,13697851,2020-02-14 15:35:07,13697851,11390804,iPhone XS Max,6625.00,13.1.3,243.0,4.0,86798.0,1.0,3.0,2.0,1.0,NaN,NaN,84,79,22,103,135,0,0,0,0,0,0,0
7,100142067,1056,13800336,13800336,2020-02-16 15:44:03,13800336,11435951,iPhone 6,4950.88,12.4.4,52.0,4.0,46767.0,1.0,8.0,1.0,1.0,NaN,NaN,8,8,6,53,36,0,0,0,0,0,0,0
8,101067630,1056,13974238,13974238,2020-02-13 18:01:47,13974238,11371861,iPhone 6s Plus,0.00,12.4.1,7.0,1.0,7.0,1.0,1.0,1.0,1.0,NaN,NaN,0,0,0,6,0,0,0,0,0,0,0,0


In [7]:
role_info.sort_values('pay_sum',ascending=False)

,user_id,mgame_id,cp_server_no,cp_role_id,create_role_time,role_name,role_id,p_model,model_money,sys_ver,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,device_num,city_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_grade_7
3533541,95961279,1056,660066,13193835,2019-10-15 02:18:49,快乐小马,142683453,ONEPLUS A5000,3149.00,9,78.0,8.0,125518.0,189.0,10.0,2.0,3.0,142.0,54490.0,137,16,64,110,74,1,16,25,17,6,77,0
2319418,116598373,1056,S987,18697005,2020-07-09 09:52:58,18697005,13717945,iPhone 8,2928.15,13.5.1,126.0,4.0,142785.0,65.0,11.0,2.0,1.0,124.0,44292.0,104,124,63,129,129,1,15,22,15,13,58,0
1608287,109006999,1056,S572,16470106,2020-03-29 14:26:37,16470106,12443719,"iPhone12,5",0.00,13.1.2,81.0,4.0,64516.0,65.0,10.0,1.0,1.0,94.0,31250.0,62,15,18,99,46,0,13,18,14,9,40,0
1518145,96051328,1056,660066,13204717,2019-10-17 01:13:13,小马快乐,136866447,SM-G9550,3599.00,26,20.0,3.0,57043.0,65.0,3.0,1.0,1.0,100.0,30298.0,65,1,24,65,19,1,13,19,19,8,40,0
3155653,105118998,1056,660373,15134986,2020-02-29 20:28:09,二嫂,11775192,iPhone X,6019.95,13.3.1,97.0,4.0,123264.0,1.0,6.0,1.0,1.0,87.0,29990.0,62,82,34,53,145,1,11,15,14,6,40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6043715,116883049,1056,S999,18770999,2020-07-12 14:41:14,18770999,13754308,iPhone XS Max,6625.00,13.5.1,2.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,0,0,2,0,0,0,0,0,0,0,0,0
6043716,116882895,1056,S999,18771016,2020-07-12 14:42:12,18771016,13754317,iPhone XS Max,6625.00,13.5.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0
6043717,116883965,1056,S999,18771203,2020-07-12 14:47:36,18771203,13754388,iPhone 8 Plus,3249.71,13.5.1,4.0,1.0,0.0,5.0,1.0,1.0,1.0,NaN,NaN,0,0,4,0,0,0,0,0,0,0,0,0
6043718,116885483,1056,S999,18771607,2020-07-12 14:59:23,18771607,13754540,"iPhone12,1",0.00,13.5.1,4.0,1.0,0.0,5.0,1.0,1.0,1.0,NaN,NaN,0,0,1,3,0,0,0,0,0,0,0,0


In [ ]:
role_info[role_info['pay_sum']]

In [9]:
from pyecharts.charts import Bar
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType

c = (
    Bar({"theme": ThemeType.MACARONS})
    .add_xaxis(Faker.choose())
    .add_yaxis("商家A", Faker.values())
    .add_yaxis("商家B", Faker.values())
    .set_global_opts(
        title_opts={"text": "Bar-通过 dict 进行配置", "subtext": "我也是通过 dict 进行配置的"}
    )
    .render("bar_base_dict_config.html")
)

In [8]:
import pyg2plot

In [5]:
# 删除没有心跳数据的样本(1、没有登录;2、没有拿到心跳)
role_info.dropna(subset=['role_created_login_num'], inplace=True)
role_info.dropna(subset=['role_created_online'], inplace=True)
role_info['pay_num'].fillna(0, inplace=True)
role_info['pay_sum'].fillna(0, inplace=True)
role_info['role_created_active'] = role_info['role_created_active'].clip(0,3)
role_info['pay_rate'] = role_info['pay_num'] / (role_info['role_created_active'] + 1e-4)
role_info['pay_avg'] = role_info['pay_sum'] / (role_info['pay_num'] + 1e-4)

In [6]:
# 清洗设备金额
role_info.loc[role_info['p_model'] == 'iPhone 6s Plus','model_money'] = 3500
role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 5000
role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 2500
role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 2000
role_info['model_money'] = role_info['model_money'].replace(0, np.nan)
role_info['model_money'].fillna(role_info[role_info['model_money'] !=0]['model_money'].mean(),inplace=True)
bins=[0,1000,1500,2000,2500,3000,3500,4000,4500,5000,6000,8000,10000]
role_info['model_money_level'] = pd.cut(role_info['model_money'],bins,labels=False)

In [8]:
# 清洗login_num
# 截取3登陆次数大于200的样本
role_info['role_created_login_num'].clip(1,120,inplace=True)
tmp = role_info[role_info['role_created_login_num'] <2]
tmp['role_created_login_num'] = tmp['max_role_level'] * 0.6
role_info['role_created_login_num'].update(tmp['role_created_login_num'])
role_info['role_created_login_num'].clip(1,120,inplace=True)

In [9]:
# 清洗max_role_level
# 按前3天在线时长更新等级小于20级的
tmp = role_info[role_info['max_role_level'] <20]
tmp['max_role_level'] = tmp['role_created_online'] * 0.0011
role_info['max_role_level'].update(tmp['max_role_level'])
# 截取等级大于65级的
role_info['max_role_level'].clip(1,65,inplace=True)

In [10]:
# 清洗online
# 按等级补全在线时长
tmp = role_info[role_info['role_created_online'] <1800]
tmp['role_created_online'] = tmp['max_role_level'] * 907
role_info['role_created_online'].update(tmp['role_created_online'])
# 截断 最小5分钟，最大24小时
role_info['role_created_online'].clip(300,86400,inplace=True)

In [11]:
# 处理时间特征create_role_time
role_info = role_info.sort_values('create_role_time').reset_index(drop=True)
role_info['hour'] = role_info['create_role_time'].dt.hour
role_info['weekend'] = role_info['create_role_time'].dt.weekday
holidays = ['2019-09-13','2019-09-14','2019-09-15','2019-10-01','2019-10-02','2019-10-03','2019-10-04','2019-10-05','2019-10-06','2019-10-07',
            '2020-01-01','2020-01-24','2020-01-25','2020-01-26','2020-01-27','2020-01-28','2020-01-29','2020-01-30','2020-04-04','2020-04-05',
            '2020-04-06','2020-04-04','2020-05-01','2020-05-02','2020-05-03','2020-05-04','2020-05-05','2020-06-25','2020-06-27']
role_info['timestamp'] = role_info['create_role_time'].dt.date.apply(lambda x:str(x))
role_info['is_holidays'] = (role_info['timestamp'].isin(holidays)).astype(int)
role_info.drop(['timestamp'], axis=1, inplace=True)

In [12]:
# 合并role_user
user_info = pd.read_csv('./user_info.csv')
user_info.drop_duplicates(inplace=True)
data = pd.merge(role_info,user_info,on='user_id',how='left',validate='many_to_one')
# 标记用户时否绑定了手机，绑定;1    未绑定：0
data['mobile'] = pd.to_numeric(data['mobile'],errors='coerce')
data['mobile'] = data['mobile'].apply(lambda x:0 if np.isnan(x) == True else 1)
# 用户注册时间与角色注册时间的时间差,并进行分桶
data['reg_date'] = pd.to_datetime(data['reg_date'])
data['time_interval'] = (data['create_role_time'] - data['reg_date']).dt.total_seconds()/3600
data['time_interval'] = data['time_interval'].clip(0,2000)
data['time_interval'] = pd.cut(data['time_interval'], bins=[0,1,3,12,24,72,168,720,2000],labels=False)

data['active_0-8'] = pd.to_numeric(data['active_0-8'], errors='coerce')
data['active_8-12'] = pd.to_numeric(data['active_8-12'], errors='coerce')
data['active_12-14'] = pd.to_numeric(data['active_12-14'], errors='coerce')
data['active_14-18'] = pd.to_numeric(data['active_14-18'], errors='coerce')
data['active_18-24'] = pd.to_numeric(data['active_18-24'], errors='coerce')

In [13]:
data['pay_grade_1'] = pd.to_numeric(data['pay_grade_1'], errors='coerce')
data['pay_grade_2'] = pd.to_numeric(data['pay_grade_2'], errors='coerce')
data['pay_grade_3'] = pd.to_numeric(data['pay_grade_3'], errors='coerce')
data['pay_grade_4'] = pd.to_numeric(data['pay_grade_4'], errors='coerce')
data['pay_grade_5'] = pd.to_numeric(data['pay_grade_5'], errors='coerce')
data['pay_grade_6'] = pd.to_numeric(data['pay_grade_6'], errors='coerce')
data['pay_grade_7'] = pd.to_numeric(data['pay_grade_7'], errors='coerce')
role_created_30_pay_sum = pd.read_csv('./data/role_created_30_pay_sum.csv',index_col=0)
# 对连接的关键字进行字符化或者整型化的调整
role_created_30_pay_sum['user_id'] = role_created_30_pay_sum['user_id'].astype(str)
role_created_30_pay_sum['mgame_id'] = role_created_30_pay_sum['mgame_id'].astype(str)
role_created_30_pay_sum['cp_server_no'] = role_created_30_pay_sum['cp_server_no'].astype(str)
role_created_30_pay_sum['cp_role_id'] = role_created_30_pay_sum['cp_role_id'].astype(str)
data['user_id'] = data['user_id'].astype(str)
data['mgame_id'] = data['mgame_id'].astype(str)
data['cp_server_no'] = data['cp_server_no'].astype(str)
data['cp_role_id'] = data['cp_role_id'].astype(str)
role_created_30_pay_sum.drop_duplicates(subset=['user_id','cp_server_no','mgame_id','cp_role_id'],inplace=True)
data.drop_duplicates(subset=['user_id','cp_server_no','mgame_id','cp_role_id'],inplace=True)
data = pd.merge(data,role_created_30_pay_sum,on=['user_id','mgame_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [14]:
data.to_pickle('./data_3d.pickle')

In [15]:
del role_info,tmp,data,role_created_30_pay_sum,user_info
gc.collect()

59